In [2]:
# %pylab notebook
import numpy as np 
import h5py
import matplotlib.pyplot as plt
%matplotlib ipympl

In [3]:
'''
column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h)
'''
# dgroup_3 = np.loadtxt('../odata3/CLAUDS_HSC_iband_group')
# d2_3 = np.loadtxt('../odata3/iCLAUDS_HSC_iband_2')
# digal_3 = np.loadtxt('../odata3/CLAUDS_HSC_iband_igal')

digal0_3 = np.loadtxt('../odata3/CLAUDS_HSC_iband_new_igal0')

In [4]:
# dgroup_1 = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
# d2_1 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')
# digal_1 = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')

digal0_1 = np.loadtxt('../odata/CLAUDS_HSC_iband_igal0')

In [13]:
digal0_1.shape

(5607052, 8)

In [5]:
def get_Vmax(cosmo_para, zgal, abMgal, zlim, mlim, galarea):
    from astropy.cosmology import FlatLambdaCDM
    import numpy as np 

    cosmo = FlatLambdaCDM(H0 = cosmo_para['H0'], Om0 = cosmo_para['Om0'])
    h = cosmo_para['H0'] / 100
    Vratio = galarea / ((180/np.pi)**2*4*np.pi)

    dlimlow = cosmo.comoving_distance(zlim[0]).value #Mpc
    Vlowmax = 4/3*np.pi*dlimlow**3 * Vratio

    abMgal = abMgal + 5*np.log10(0.674)
    dmax = 10**((mlim - abMgal - 25) / 5) #maximum distance from limited luminosity
    dmax_como = dmax / (1 + zgal)

    dlimup = cosmo.comoving_distance(zlim[1]).value
    Vupmax = np.zeros(zgal.shape[0])
    Vupmax[dmax_como < dlimup] = 4/3*np.pi*(dmax_como[dmax_como < dlimup])**3 * Vratio
    Vupmax[dmax_como >= dlimup] = 4/3*np.pi*(dlimup)**3 * Vratio

    Vmax = Vupmax - Vlowmax
    return Vmax

In [6]:
def cal_galarea(ra,dec):
    import healpy as hp
    import pandas as pd

    nside = 512
    totalpixel = hp.nside2npix(nside)

    pxl = hp.ang2pix(nside, np.pi/2.-np.deg2rad(dec), np.deg2rad(ra))
    pxlc = pd.value_counts(pxl)

    gal_area = pxlc.shape[0] / totalpixel * (4*180**2/np.pi)
    print("the cover of galaxy region is %s dge2" %gal_area)

    return gal_area

In [37]:
def cal_Vmax(zbin,digal0):
    from astropy.cosmology import FlatLambdaCDM
    import numpy as np
    cosmo_para = {'H0':67.4, 'Om0':0.315}
    cosmo = FlatLambdaCDM(H0 = cosmo_para['H0'], Om0 = cosmo_para['Om0'])

    sel_redz = np.where((digal0[:,4] >= zbin[0]) & (digal0[:,4] < zbin[1]))[0]
    zgal = digal0[sel_redz,4]
    abmgal = digal0[sel_redz,6]
    
    grpra = digal0[sel_redz,2]
    grpdec = digal0[sel_redz,3]
    grparea = cal_galarea(grpra,grpdec)
    
#     grparea = 35
    Vmax = get_Vmax(cosmo_para, zgal, abmgal, zlim = zbin, mlim = 26, galarea = grparea) * 0.674**3 #(Mpc/h)^3
#     Mag = digal0[sel_redz, -2]+ 5*np.log10(0.674)
    Mag = abmgal

    return Mag,Vmax

In [32]:
#Schechter function for absolute magnitude
def SF_M(M, phi, Ms, alpha):
    deltaM = Ms - M
    a1 = np.log(10)/2.5*phi
    a2 = (10**(0.4*deltaM))**(alpha+1)
    a3 = np.exp(-10**(0.4*deltaM))
    return a1*a2*a3

def plot_ob():
    Mgal = np.linspace(-24, -16, 100)
    
#     #Blanton+2001 (h = 1)
#     phi = 1.28*1e-2 #h3Mpc-3
#     Ms = -21.26
#     alpha = -1.25   
#     Lfunc = SF_M(Mgal, phi, Ms, alpha)
#     plt.plot(Mgal, Lfunc, ls = '--', c = 'k', label = 'Blanto et al. 2001')
    
    #Blanton+2003 (h = 1)
    phi = 1.47*1e-2 #h3Mpc-3
    Ms = -20.82
    alpha = -1.00   
    Lfunc = SF_M(Mgal, phi, Ms, alpha)
    plt.plot(Mgal, Lfunc, ls = '-', c = 'k', label = 'Blanto+2003(0.02 < z < 0.22)')
    
    #Hill+2010
    phi = 0.0120
    Ms = -21.16
    alpha = -1.18
    Lfunc = SF_M(Mgal, phi, Ms, alpha)
    plt.plot(Mgal, Lfunc, ls = '--', c = 'k', label = 'Hill+2010(z < 0.1)')
    
#     #Kelvin+2014
#     phi = 3.61*1e-3
#     Ms = -22.15
#     alpha = -1.14
#     Lfunc = SF_M(Mgal, phi, Ms, alpha)
#     plt.plot(Mgal, np.log10(Lfunc), ls = '-', c = 'b', label = 'Kelvin et al. 2014')
    
    #Drive+2012
    phi = 1*1e-2 #1e−2h3Mpc−3 
    Ms = -21.30
    alpha = -1.17    
    Lfunc = SF_M(Mgal, phi, Ms, alpha)
    plt.plot(Mgal, Lfunc, ls = ':', c = 'k', label = 'Drive+2012 (0.013 < z < 0.1)')

def plot_ob2():
    #Bates+2018
    xx = np.array([-25.125, -24.875, -24.625, -24.375, -24.125, -23.875, -23.625]) - 5*np.log10(0.674)
    yy = np.array([0.001, 0.005, 0.018, 0.033, 0.124, 0.229, 0.388]) / 0.674**3 * 1e-3
    yyerr = np.array([0.014,0.015,0.024,0.018,0.026,0.023,0.036]) / 0.674**3 * 1e-3
    cbar = plt.errorbar(xx, yy ,yerr = yyerr, marker = '^', c = 'k', ls = 'none', ms = 5,
                 capsize = 2, label = 'Bates+2019 (0.7 < z < 0.8)')
    return cbar

In [19]:
#plot galaxy luminosity function
def plot_LF(digal0):
    colors = ['k','purple','blue','cyan','magenta','red','orange']

    xmin = -28
    xmax = -16
    nbin = 20
    xlum = np.linspace(xmin,xmax,nbin+1)
    nx = (xmax-xmin)/nbin/2
    xx = xlum[:-1]+nx

    zbinall = [[0.0,0.4],[0.8,1.2],[1.6,2.0],[2.4,2.8],[3.6,4.0],[4.8,5.2],[5.6,6.0]]
    for i in range(7):
        zbin = zbinall[i]
            
        Mag, Vmax = cal_Vmax(zbin,digal0)
        ngroup,xe = np.histogram(Mag, bins=xlum, weights=1/Vmax)
        ng,xe = np.histogram(Mag, bins=xlum)
        yy = ngroup/(nx*2)
        
        yyerr,xe = np.histogram(Mag, bins=xlum, weights=1/Vmax**2)
        yyerr = np.sqrt(yyerr)

        idx = np.where(yy > 0)[0] 
        plt.errorbar(xx[idx], yy[idx], yerr = yyerr[idx], capsize = 2.0, 
                    ms = 5, mew = 1, marker = 'o', color = colors[i],
                     mfc = 'none', elinewidth = 1,
                     ls = 'none', label = '%s $\leq$ z < %s' %(zbin[0],zbin[1]))

    plot_ob() 
    
    plt.xlim(-28,-16)
    plt.ylim(1e-8,1e-1)
    plt.yscale('log')
    ax = plt.gca()
    ax.invert_xaxis()

    plt.ylabel(r'$\Phi [\mathrm{mag^{-1}}h^3\mathrm{Mpc^{-3}}]$', fontsize = 14)
    plt.tick_params(top = 'on', right = 'on',which='both', direction = 'in', labelsize = 12)

    plt.tight_layout()

In [10]:
def plot_Mz(digal0):
    import numpy as np
    
    zgal = digal0[:,4] #redshift
    abmgal = digal0[:,6] #absolute magnitude

    plt.plot(zgal, abmgal, '.', ms = 0.01, rasterized=True)

    ax = plt.gca()
    ax.invert_yaxis()
    
    plt.xlabel(r'$z_{\rm photo}$', fontsize = 14)
    plt.ylim(-9,-28)

In [41]:
#make a combined comparision
plt.figure(figsize = (10,8))
# plt.figure(figsize = (10,8))
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally

plt.subplot(221)
plot_LF(digal0_1)
plt.xlabel(r'$M_{i} - 5\log h$', fontsize = 14)
plt.text(-23.4,1e-2, 'no $K$-correction', fontsize = 12, fontweight = 350)
plt.legend(ncol = 1, fontsize = 'x-small', loc = 3)

plt.subplot(222)
plot_Mz(digal0_1)
plt.ylabel(r'$M_{i} - 5\log h$', fontsize = 14)
plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 12)

plt.subplot(223)
plot_LF(digal0_3)
cbar = plot_ob2()
plt.xlabel(r'$M_{i}^{0.0} - 5\log h$', fontsize = 14)
plt.text(-23.8,1e-2, '$K$-correction', fontsize = 12, fontweight = 350)
plt.legend(handles = [cbar], fontsize = 'x-small', loc = 3)

plt.subplot(224)
plot_Mz(digal0_3)
plt.ylabel(r'$M^{0.0}_{i} - 5\log h$', fontsize = 14)
plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 12)

plt.tight_layout()

plt.savefig('../figs/LF2_gal.pdf', dpi = 400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the cover of galaxy region is 35.17164931963046 dge2
the cover of galaxy region is 35.11919346680476 dge2
the cover of galaxy region is 34.83068627626342 dge2
the cover of galaxy region is 34.84380023946984 dge2
the cover of galaxy region is 34.0176205574651 dge2
the cover of galaxy region is 25.441088620463493 dge2
the cover of galaxy region is 4.458747490184323 dge2
the cover of galaxy region is 35.17164931963046 dge2
the cover of galaxy region is 35.11919346680476 dge2
the cover of galaxy region is 34.83068627626342 dge2
the cover of galaxy region is 34.84380023946984 dge2
the cover of galaxy region is 34.0176205574651 dge2
the cover of galaxy region is 25.441088620463493 dge2
the cover of galaxy region is 4.458747490184323 dge2


In [74]:
# def cal_galarea0(ra,dec,nside):
#     import healpy as hp
#     import pandas as pd

#     totalpixel = hp.nside2npix(nside)

#     pxl = hp.ang2pix(nside, np.pi/2.-np.deg2rad(dec), np.deg2rad(ra))
#     pxlc = pd.value_counts(pxl)

#     gal_area = pxlc.shape[0] / totalpixel * (4*180**2/np.pi)
#     print("the cover of galaxy region is %s dge2" %gal_area)

#     return gal_area

In [82]:
# sel_redz = np.where((digal0_1[:,4] >= 0) & (digal0_1[:,4] < 0.4))[0]


# grpra = digal0_1[sel_redz,2]
# grpdec = digal0_1[sel_redz,3]

# for i in range(8):
#     grparea = cal_galarea0(grpra,grpdec,nside = i*200+200)

the cover of galaxy region is 41.08107391088002 dge2
the cover of galaxy region is 36.11782701055927 dge2
the cover of galaxy region is 34.38701700443491 dge2
the cover of galaxy region is 33.50728805649445 dge2
the cover of galaxy region is 32.86142138193323 dge2
the cover of galaxy region is 32.341080210988586 dge2
the cover of galaxy region is 31.93245801475031 dge2
the cover of galaxy region is 31.49701191748467 dge2


In [ ]:
#test for observations

# plt.figure()

# plot_ob()

# # B03 = np.loadtxt(open('../data/ob/Hill+2010/Blanton03_i.csv', 'r'),delimiter = ',')
# # plt.plot(B03[:,0], B03[:,1], c = 'r', ls = ':', label = 'B03', lw =2 , zorder = -1)

# # H10 = np.loadtxt(open('../data/ob/Hill+2010/Hill_i.csv', 'r'),delimiter = ',')
# # plt.plot(H10[:,0], H10[:,1], c = 'r', ls = '-', label = 'H10', lw =2 , zorder = -1)

# # plt.yscale('log')
# ax = plt.gca()
# ax.invert_xaxis()

# plt.ylim(-6,-1)

# plt.grid(c = 'grey' ,ls = '--', alpha = 0.6)
# plt.legend()